In [1]:
from nested_cv import NestedCV

import pandas as pd
import numpy as np
from sklearn.datasets import load_boston, load_iris, load_breast_cancer
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier

# When using Random Search, we get a user warning with this little number of hyperparameters
# Suppress it
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

# Regression Example

In [2]:
boston = load_boston()
X = boston.data
y = boston.target

# Define a parameters grid
param_grid = {
     'max_depth': [3, None],
     'n_estimators': [10]
}

NCV = NestedCV(model=RandomForestRegressor(), params_grid=param_grid, outer_kfolds=5, inner_kfolds=5,
               cv_options={'sqrt_of_score':True, 'randomized_search_iter':30,
                           'recursive_feature_elimination':True, 'rfe_n_features':2})
NCV.fit(X=X,y=y)

NCV.outer_scores

[4.316508190705864,
 4.44304508905537,
 4.278287426753842,
 3.8147803500361337,
 4.8574557263467595]

# Classification Example

## Breast Cancer (2 Classes)

In [3]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

# Binary classification
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target

# Define a parameters grid
param_grid = {
     'max_depth': [3, None],
     'n_estimators': [10, 20]
}

NCV = NestedCV(model=RandomForestClassifier(), params_grid=param_grid, outer_kfolds=5, inner_kfolds=5,
               cv_options={'metric':roc_auc_score, 'metric_score_indicator_lower':False,
                           'randomized_search_iter':30, 'predict_proba':True})
NCV.fit(X=X,y=y)

NCV.outer_scores

[0.9934490664919751,
 0.9952922077922078,
 0.972972972972973,
 0.9912087912087912,
 0.9991337491337491]

## Iris (3 Classes)

In [4]:
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV

# Multiclass classification
iris = load_iris()
X = iris.data
y = iris.target

# Define a parameters grid
param_grid = {
     'max_depth': [3, None],
     'n_estimators': [10, 20]
}

NCV = NestedCV(model=RandomForestClassifier(), params_grid=param_grid, outer_kfolds=5, inner_kfolds=5,
               cv_options={'metric':roc_auc_score, 'metric_score_indicator_lower':False,
                           'randomized_search_iter':30, 'predict_proba':True})
NCV.fit(X=X,y=y)

NCV.outer_scores

ValueError: multiclass is not support yet in nested cross-validation